## Preparation

In [32]:
import torch
import torch.nn as nn

In [33]:
# Set path
import pickle
path = 'C:/Users/User/Desktop/Assignatures/Synthesis project/GraphAnomaly/dades_arnau/'

In [34]:
# Load train and test set
with open(path + 'sequential_train.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open(path + 'sequential_test.pkl', 'rb') as f:
    test_set = pickle.load(f)

In [35]:
from Helper import separate_labels

# Split labels
X_train, y_train, X_test, y_test = separate_labels(train_set, test_set)

X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

In [186]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=167, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [97]:
from torch.utils.data import Dataset
class TargetDataset(Dataset):
    def __init__(self, X, y):
        self.X = X 
        self.y = y       

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):        
        return self.X[idx], self.y[idx]

In [185]:
from torch.utils.data import DataLoader

train_loader = DataLoader(TargetDataset(X_train, y_train), batch_size=128, shuffle=True)
test_loader = DataLoader(TargetDataset(X_test, y_test), batch_size=128, shuffle=False)

In [167]:
def train(model, train_loader, criterion, optimizer, num_epochs=10): 
    total_train_loss = [] 
    # total_validation_loss = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for input, target in train_loader:
            optimizer.zero_grad()
            
            # Forward pass
            output = model(input.permute(1,0))
            
            # Compute the loss
            loss = criterion(output, target.float())            
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}")
        total_train_loss.append(train_loss)
        
        # #validació dins del mateix train loop
        # validation_loss = val_ae(autoencoder=autoencoder, data_loader=val_loader, criterion=criterion)
        # print(f"Validation Loss: {validation_loss:.4f}")
        # total_validation_loss.append(validation_loss)
    return total_train_loss#, total_validation_loss

In [183]:
lr = 0.001
EPOCHS = 10
model = CNNBinaryClassifier(167)
criterion = nn.BCELoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

l = train(model, train_loader, criterion, optimizer, EPOCHS)

RuntimeError: running_mean should contain 16 elements not 64

In [141]:
def infer(model, loader, criterion):
    model.eval() 
    total_loss = []
    y_pred = []    
    val_loss = 0.0

    with torch.no_grad():
        for input, target in loader:
            output = model(input)
            y_pred.append(output)
            loss = criterion(output, target.float())            
            val_loss += loss.item()
    validation_loss = val_loss / len(loader.dataset)    
    print( 'Test Loss: ' + str(round(validation_loss,4)))
    total_loss.append(validation_loss)
    return total_loss, y_pred

In [142]:
loss, y_pred = infer(model, test_loader, criterion)

Test Loss: 0.0021


In [144]:
prediction = []
for i in range(len(y_pred)):
    y_pred[i] = y_pred[i].detach().numpy().tolist()
    prediction += y_pred[i]

In [149]:
len(y_test)

16670

In [150]:
co = 0
inco = 0

for i in range(len(prediction)):
    if prediction[i] < 0.92:
        if y_test[i]==0:
            co+=1
        else:
            inco+=1
    else:
        if y_test[i]==1:
            co+=1
        else:
            inco+=1

In [153]:
inco

12014